In [75]:
import pandas as pd
import numpy as np
import csv
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [110]:
# Data pre-processing And Feature Sellection

# Read data
covidData = pd.read_csv("./data/owid-covid-data.csv")
# Drop not wanted columns
covidData.drop(["continent","iso_code","human_development_index","new_deaths_smoothed","total_cases_per_million","new_cases_per_million","new_cases_smoothed_per_million","gdp_per_capita","extreme_poverty","female_smokers","male_smokers","handwashing_facilities","life_expectancy"],axis=1,inplace=True)
covidData.drop(["total_cases","new_cases_smoothed","total_deaths_per_million","new_deaths_per_million","new_deaths_smoothed_per_million","reproduction_rate","new_tests_smoothed","new_tests_smoothed_per_thousand","positive_rate","tests_per_case","stringency_index","median_age","aged_65_older"],axis=1,inplace=True)
covidData.drop(["total_deaths","icu_patients","icu_patients_per_million","hosp_patients","hosp_patients_per_million","weekly_icu_admissions","weekly_icu_admissions_per_million","weekly_hosp_admissions","weekly_hosp_admissions_per_million","total_tests","new_tests","total_tests_per_thousand","new_tests_per_thousand","tests_units"],axis=1,inplace=True)
covidData.drop(["total_vaccinations","people_vaccinated","people_fully_vaccinated","new_vaccinations","new_vaccinations_smoothed","total_vaccinations_per_hundred","people_vaccinated_per_hundred","people_fully_vaccinated_per_hundred","new_vaccinations_smoothed_per_million"],axis=1,inplace=True)

covidData = pd.DataFrame(covidData)
covidData = covidData.fillna(0)

# Dropping row 'Wrold',and row 'International' considered to be of no use 
covidData.drop(covidData[covidData['location'] == 'World'].index,inplace=True)
covidData.drop(covidData[covidData['location'] == 'International'].index,inplace=True)
# Dropping row 'Hong Kong' due to data absence of data
covidData.drop(covidData[covidData['location'] == 'Hong Kong'].index,inplace=True)

trainOlder70 = []
trainMonths = []
trainCases = {}
trainDeaths = {}
trainPopulation=[]
trainPopulationDens = []
trainCardiovascDeathRate = []
trainDiabetesPrevalence = []
trainHospitalBedsPer_k = []

testOlder70 = []
testMonths = []
testCases = {}
testDeaths = {}
testPopulation=[]
testPopulationDens = []
testCardiovascDeathRate = []
testDiabetesPrevalence = []
testHospitalBedsPer_k = []

# Gather features and store them in dataframes
targetCountries = ['Greece','Cuba','United Kingdom','Canada','Kenya','Afghanistan','Israel','Australia','Argentina',"Italy"]
for index,row in covidData.iterrows():
    if row["location"] not in targetCountries:
        if int(row["date"].split("-")[0]) < 2021:
            if row["location"] not in trainCases:
                trainCardiovascDeathRate.append(row["cardiovasc_death_rate"])
                trainDiabetesPrevalence.append(row["diabetes_prevalence"])
                trainPopulation.append(row["population"])
                trainPopulationDens.append(row["population_density"])
                trainOlder70.append(row["aged_70_older"])
                trainHospitalBedsPer_k.append(row["hospital_beds_per_thousand"])
                trainCases[row["location"]] = {int(row["date"].split("-")[1]): int(row['new_cases'])}
                trainCases[row["location"]] = {13: row['new_cases']}
                trainDeaths[row["location"]] = {int(row["date"].split("-")[1]): int(row['new_deaths'])}
                trainDeaths[row["location"]] = {13: row['new_deaths']}
            else:
                if int(row["date"].split("-")[1]) in trainCases[row["location"]]:
                    trainCases[row["location"]][int(row["date"].split("-")[1])] += int(row['new_cases'])
                else:
                    trainCases[row["location"]][int(row["date"].split("-")[1])] = int(row['new_cases'])
                if int(row["date"].split("-")[1]) in trainDeaths[row["location"]]:
                    trainDeaths[row["location"]][int(row["date"].split("-")[1])] += int(row['new_deaths'])
                else:
                    trainDeaths[row["location"]][int(row["date"].split("-")[1])] = int(row['new_deaths'])
                trainCases[row["location"]][13] += int(row['new_cases'])
                trainDeaths[row["location"]][13] += int(row['new_deaths'])
    else:    
        currentMonth = int(row["date"].split("-")[1])
        if int(row["date"].split("-")[0]) == 2021:
            currentMonth += 12
        
        if row["location"] not in testCases:
            testCardiovascDeathRate.append(row["cardiovasc_death_rate"])
            testDiabetesPrevalence.append(row["diabetes_prevalence"])
            testPopulation.append(row["population"])
            testPopulationDens.append(row["population_density"])
            testOlder70.append(row["aged_70_older"])
            testHospitalBedsPer_k.append(row["hospital_beds_per_thousand"])
            testCases[row["location"]] = {currentMonth: int(row['new_cases'])}
            testCases[row["location"]] = {14: row['new_cases']}
            testDeaths[row["location"]] = {currentMonth: int(row['new_deaths'])}
            testDeaths[row["location"]] = {14: row['new_deaths']}
        else:
            if currentMonth in testCases[row["location"]]:
                testCases[row["location"]][currentMonth] += int(row['new_cases'])
            else:
                testCases[row["location"]][currentMonth] = int(row['new_cases'])
            if currentMonth in testDeaths[row["location"]]:
                testDeaths[row["location"]][currentMonth] += int(row['new_deaths'])
            else:
                testDeaths[row["location"]][currentMonth] = int(row['new_deaths'])

            testCases[row["location"]][14] += int(row['new_cases'])
            testDeaths[row["location"]][14] += int(row['new_deaths'])


# Train dataframe
trainCovidData = pd.DataFrame()
trainCovidData["country"] = list(trainCases.keys())
# Gathering the number of monthly cases and deaths for each country
for i in range(13):
    monthlyCases = []
    monthlyDeaths = []
    for country,months in trainCases.items():
        monthWithoutCases = True
        for month,cases in months.items():
            if month == i + 1:
                monthlyCases.append(cases)
                monthWithoutCases = False
                break
        if monthWithoutCases:
            monthlyCases.append(0)
    
    for country,months in trainDeaths.items():
        monthWithoutDeaths = True
        for month, deaths in months.items():
            if month == i + 1:
                monthlyDeaths.append(deaths)
                monthWithoutDeaths = False
                break
        if monthWithoutDeaths:
            monthlyDeaths.append(0)

    if i + 1 != 13:
        trainCovidData["casesforMonth-"+str(i+1)] = monthlyCases
    else:
        trainCovidData["totalCases"] = monthlyCases
        
    if i + 1 != 13:
        trainCovidData["deathsforMonth-"+str(i+1)] = monthlyDeaths
    else:
        trainCovidData["totalDeaths"] = monthlyDeaths

    monthlyCases.clear()
    monthlyDeaths.clear()
trainCovidData["cardiovascularDeathRate"] = trainCardiovascDeathRate
trainCovidData["diabetes"] = trainDiabetesPrevalence
trainCovidData["population"] = trainPopulation
trainCovidData["populationDensity"] = trainPopulationDens
trainCovidData["older70"] = trainOlder70
trainCovidData[u"beds‰"] = trainHospitalBedsPer_k


# Test dataframe
testCovidData = pd.DataFrame()
testCovidData["country"] = list(testCases.keys())
for i in range(14):
    monthlyCases = []
    monthlyDeaths = []
    for country,months in testCases.items():
        monthWithoutCases = True
        for month,cases in months.items():
            if month == i + 1:
                monthlyCases.append(cases)
                monthWithoutCases = False
                break
        if monthWithoutCases:
            monthlyCases.append(0)
    
    for country,months in testDeaths.items():
        monthWithoutDeaths = True
        for month,deaths in months.items():
            if month == i + 1:
                monthlyDeaths.append(deaths)
                monthWithoutDeaths = False
                break
        if monthWithoutDeaths:
            monthlyDeaths.append(0)
    
    if i + 1 != 14:
        testCovidData["casesforMonth-"+str(i+1)] = monthlyCases
    else:
        testCovidData["totalCases"] = monthlyCases

    if i + 1 != 14:
        testCovidData["deathsforMonth-"+str(i+1)] = monthlyDeaths
    else:
        testCovidData["totalDeaths"] = monthlyDeaths

    monthlyCases.clear()
    monthlyDeaths.clear()
testCovidData["cardiovascularDeathRate"] = testCardiovascDeathRate
testCovidData["diabetes"] = testDiabetesPrevalence
testCovidData["population"] = testPopulation
testCovidData["populationDensity"] = testPopulationDens
testCovidData["older70"] = testOlder70
testCovidData[u"beds‰"] = testHospitalBedsPer_k


display(trainCovidData)
display(testCovidData)

,country,casesforMonth-1,deathsforMonth-1,casesforMonth-2,deathsforMonth-2,casesforMonth-3,deathsforMonth-3,casesforMonth-4,deathsforMonth-4,casesforMonth-5,...,casesforMonth-12,deathsforMonth-12,totalCases,totalDeaths,cardiovascularDeathRate,diabetes,population,populationDensity,older70,beds‰
0,Albania,0,0,0,0,241,15,530,16,364,...,20134,371,58316.0,1181.0,304.195,10.08,2877800.0,104.871,8.643,2.89
1,Algeria,0,0,0,0,715,44,3290,406,5388,...,16411,325,99610.0,2756.0,278.364,6.73,43851043.0,17.348,3.857,1.90
2,Andorra,0,0,0,0,375,12,369,30,19,...,1304,8,8049.0,84.0,109.135,7.97,77265.0,163.755,0.000,0.00
3,Angola,0,0,0,0,6,2,20,0,59,...,2414,57,17553.0,405.0,276.045,3.94,32866268.0,23.890,1.362,0.00
4,Antigua and Barbuda,0,0,0,0,6,0,17,3,2,...,18,1,159.0,5.0,191.511,13.17,97928.0,231.845,4.631,3.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,Venezuela,0,0,0,0,133,3,198,13,1177,...,11164,131,113558.0,1028.0,204.850,6.47,28435943.0,36.253,3.915,0.80
175,Vietnam,0,0,14,0,196,0,58,0,58,...,118,0,1465.0,35.0,245.465,6.00,97338583.0,308.127,4.718,2.60
176,Yemen,0,0,0,0,0,0,5,2,317,...,22,5,2099.0,610.0,495.003,5.35,29825968.0,53.508,1.583,0.70
177,Zambia,0,0,0,0,33,0,71,3,951,...,3078,31,20725.0,388.0,234.499,3.94,18383956.0,22.995,1.542,2.00


,country,casesforMonth-1,deathsforMonth-1,casesforMonth-2,deathsforMonth-2,casesforMonth-3,deathsforMonth-3,casesforMonth-4,deathsforMonth-4,casesforMonth-5,...,casesforMonth-13,deathsforMonth-13,totalCases,totalDeaths,cardiovascularDeathRate,diabetes,population,populationDensity,older70,beds‰
0,Afghanistan,0,0,0,0,174,4,1952,60,13081,...,3413,208,54939.0,2399.0,597.029,9.59,38928341.0,54.422,1.337,0.50
1,Argentina,0,0,0,0,1054,27,3374,191,12423,...,289848,4530,1915362.0,47775.0,191.032,5.50,45195777.0,16.177,7.441,5.00
2,Australia,5,0,16,0,4534,18,2207,75,436,...,381,0,28806.0,909.0,107.791,5.07,25499881.0,3.202,10.129,3.84
3,Canada,3,0,16,0,8507,152,45930,4027,38022,...,190516,4039,774925.0,19801.0,105.599,7.37,37742157.0,4.037,10.797,2.50
4,Cuba,0,0,0,0,183,6,1315,55,544,...,12901,64,24764.0,210.0,190.968,8.27,11326616.0,110.408,9.719,5.20
5,Greece,0,0,4,0,1310,49,1277,91,326,...,16828,926,155678.0,5764.0,175.695,4.55,10423056.0,83.479,14.524,4.21
6,Israel,0,0,6,0,5711,21,10179,204,1183,...,210729,1375,633991.0,4700.0,93.320,6.74,8655541.0,402.606,7.359,2.99
7,Italy,0,0,1126,29,104664,12399,99671,15539,27534,...,421904,13699,2529070.0,87858.0,113.151,4.78,60461828.0,205.859,16.240,3.18
8,Kenya,0,0,0,0,59,1,337,16,1566,...,4105,83,100563.0,1753.0,218.637,2.92,53771300.0,87.324,1.528,1.40
9,United Kingdom,0,0,59,0,38754,2457,139956,24297,78768,...,1287362,30950,3783593.0,104572.0,122.137,4.28,67886004.0,272.898,12.527,2.54


In [111]:
fig = make_subplots(rows=2, cols=3)

fig.add_trace(
    go.Scatter(y=trainCovidData["totalDeaths"], x=trainCovidData[u"beds‰"],
               mode = 'markers',
               text=trainCovidData["country"],
               name="beds/deaths"),
    row=1,col=1
)

fig.add_trace(
    go.Scatter(y=trainCovidData["totalDeaths"], x=trainCovidData["older70"],
               mode = 'markers',
               text=trainCovidData["country"],
               name=">70/deaths"),
    row=1,col=2
)

fig.add_trace(
    go.Scatter(y=trainCovidData["totalDeaths"], x=trainCovidData["population"],
               mode = 'markers',
               text=trainCovidData["country"],
               name="population"),
    row=1,col=3

)

fig.add_trace(
    go.Scatter(y=trainCovidData["totalDeaths"], x=trainCovidData["populationDensity"],
               mode = 'markers',
               text=trainCovidData["country"],
               name="populationDensity"),
    row=2,col=1
)

fig.add_trace(
    go.Scatter(y=trainCovidData["totalDeaths"], x=trainCovidData["cardiovascularDeathRate"],
               mode = 'markers',
               text=trainCovidData["country"],
               name="cardiovascularDeathRate"),
    row=2,col=2
)

fig.add_trace(
    go.Scatter(y=trainCovidData["totalDeaths"], x=trainCovidData["diabetes"],
               mode = 'markers',
               text=trainCovidData["country"],
               name="diabetes"),
    row=2,col=3
)

fig.update_layout(height=1000, width=1000, title_text="Side By Side Subplots")
fig.show()